In [1]:
import os

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [2]:
%%time
import json
import pandas as pd
import numpy as np
import cv2
from PIL import Image
#from tesserocr import PyTessBaseAPI, PSM, RIL, PT
import matplotlib.pyplot as plt

CPU times: user 622 ms, sys: 481 ms, total: 1.1 s
Wall time: 41.4 s


In [3]:
%%time
import lib.utils as utils
import lib.config as config
import lib.model as model
import lib.roi as roi
import lib.split_pages as spages
import lib.utils as utils

/mnt/shared/scratch/ikarunak/apps/conda/envs/lightfootcat/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CPU times: user 2.1 s, sys: 1.24 s, total: 3.34 s
Wall time: 1min 53s


In [4]:
image_path = "images/cropped/The_Lightfoot_Herbarium_05_cropped_1.jpg"
image_dir = "images/cropped/"

In [5]:
images = sorted(utils.load_images(image_dir))[:4]

In [6]:
newprompt = """
The input image contains a cropped image of a botanical catalogue listing the contents of folders sorted and listed in taxonomic order.
These information are splitted into columns in each page.
The image provides the division name, family name, species name and their folders' contents. 
---
An example of the provided format is:

Dicotyledones
ACERACEAE
Acer campestre L.
1 folder. Acer campestre [TA]
Acer pseudoplatanus L.
2 folders.
Folder 1. Acer Pseudo-Platanus
[G]. i. "Maple. Bulls: [Bulstrode]
Park" [JL]
Folder 2. Acer Pseudo-Platanus
[TA].
AMARANTHACEAE
Amaranthus lividus L. Flora Europaea 1: 110 (1964)
1 folder. Amaranthus Blitum [TA].
i. Cities Ray's Syn. 1957. ii. "Blite 
Amaranth. Aug. It is often found
on Dunghills in the neighbourhood
of London. I gather'd this on a
Dunghill at Fulham near London"
[JL]. iii. "Amaranthus Blitum.
Monoec: 5. and:" [JL]
---

where divison names are characterised by large bold words; family names as captial words; species names as italic words.
The list of contents under each species lists the number of folders first and then their content. 
When more than one folder is provided roman (like i, ii, iii) are used to differentiate the folders.
Furthermore, for each collection the collectors initials are also cited using square brackets '[]'.

In some cases, the input image provided contains folder information from previously provided images and thus these information had continued from the last page.

---
Example of output lists of JSON dicts

[
{
'division':'Dicotyledones'
'contents': [{'family'='ACERACEAE', contents=[{'species'='Acer campestre L.', folders=[{'folder'=1, 'content'='Acer campestre [TA]'}]}, 
                                              {'species'='Acer pseudoplatanus L.', folders=[{'folder'=1, 'content'='Acer Pseudo-Platanus [G]. i. "Maple. Bulls: [Bulstrode] Park" [JL]'},
                                              {'folder'=2, 'content'='Acer Pseudo-Platanus [TA]'}]}]
},{'family'='AMARANTHACEAE', contents=[{'species'='Amaranthus lividus L. Flora Europaea 1: 110 (1964)', folders=[{'folder'=1, 'content'='Amaranthus Blitum [TA].
i. Cities Ray's Syn. 1957. ii. "Blite 
Amaranth. Aug. It is often found
on Dunghills in the neighbourhood
of London. I gather'd this on a
Dunghill at Fulham near London"
[JL]. iii. "Amaranthus Blitum.
Monoec: 5. and:" [JL]'}]}, ]}

]}

---


Process the image using the following steps to generate a JSON file that supports the above provided structure

1. Create a list of JSON files
2. Find the division name where possible, if not available set as 'N/A'.
3. Under the division, create a contents lists
4. For each family name under division name create a JSON dict and add it to the list
5. Under each family name find the species name and thier folder contents and add them to the list
6. Where appropriate if a value for a specific key cannot be found, set it as 'N/A'
7. In cases where extra text is provided before Division name, family names or species name, consider the division, family or species name as 'Extra info'

Compile these information into a list of JSON dicts and return the list. Please output this list of JSON dicts and not the example. 
The example (denoted between --- and ---) is only to be used as a template for the structure of the JSON format.


Family names are characterised by captial words whilst species names are characterised by italic words.
The list of contents under each species lists the number of folders first and then their content. 
Furthermore, for each collection the collectors initials are also cited using square brackets '[]'.
"""

In [7]:
simple_prompt = "Parse the text in the image into a JSON format"

In [8]:
%%time
qwen_model = model.QWEN_Model(prompt=simple_prompt, max_new_tokens = 5000, batch_size=1)

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:59<00:00, 23.90s/it]


CPU times: user 2.87 s, sys: 11.2 s, total: 14.1 s
Wall time: 2min 5s


In [9]:
%%time
outputs = qwen_model.batch_infer(images, save=False, debug=True)

>>> Using: 
 	Maximum new tokens = 5000 
 	Batch size = 1 
 	save_path = None
>>> Batch 1 starting...
	Processing text prompts...
	Reading Images...
	Processing inputs...
	Moving inputs to gpu...
	Performing inference...
	Inference Finished
	Seperating Ids...
	Decoding Ids...
	Seperating Outputs...
	Outputs stored!
>>> Batch 2 starting...
	Processing text prompts...
	Reading Images...
	Processing inputs...
	Moving inputs to gpu...
	Performing inference...
	Inference Finished
	Seperating Ids...
	Decoding Ids...
	Seperating Outputs...
	Outputs stored!
>>> Batch 3 starting...
	Processing text prompts...
	Reading Images...
	Processing inputs...
	Moving inputs to gpu...
	Performing inference...
	Inference Finished
	Seperating Ids...
	Decoding Ids...
	Seperating Outputs...
	Outputs stored!
>>> Batch 4 starting...
	Processing text prompts...
	Reading Images...
	Processing inputs...
	Moving inputs to gpu...
	Performing inference...
	Inference Finished
	Seperating Ids...
	Decoding Ids...
	Seper

In [10]:
outputs

[('images/cropped/The_Lightfoot_Herbarium_05_cropped_1.jpg',
  '```json\n{\n  "metadata": {\n    "division": "Dicotyledones",\n    "page": "132"\n  },\n  "contents": [\n    {\n      "familyName": "Aceraceae",\n      "species": [\n        {\n          "speciesName": "Acer campestre L.",\n          "folders": [\n            {\n              "description": "1 folder. Acer campestre [TA]",\n              "citations": ["TA"]\n            }\n          ]\n        },\n        {\n          "speciesName": "Acer pseudoplatanus L.",\n          "folders": [\n            {\n              "description": "2 folders. Folder 1. Acer Pseu-do-Platanus [G]. i. \\"Maple. Bulls: [Bulstrode] Park\\" [JL] Folder 2. Acer Pseu-do-Platanus [TA].",\n              "citations": ["G", "JL"]\n            }\n          ]\n        }\n      ]\n    },\n    {\n      "familyName": "Amaranthaceae",\n      "species": [\n        {\n          "speciesName": "Amaranthus lividus L., Flora Europaea 1: 110 (1964)",\n          "folde

In [15]:
out = outputs[2][1]

In [16]:
print(out)

```json
{
  "metadata": {
    "division": "N/a",
    "page": "134"
  },
  "contents": [
    {
      "familyName": "Cucubalaceae",
      "species": [
        {
          "speciesName": "Cucubalus baccifer L.",
          "folders": [
            {
              "description": "3 folders and 1 sheet.",
              "citations": [
                "G",
                "JL"
              ]
            },
            {
              "description": "Folder 1. Cucubalus baccifer [G]. i. \"Bulstrode\" [JL].",
              "citations": [
                "G",
                "JL"
              ]
            },
            {
              "description": "Folder 2. Cucubalus baccifer [G]. i. \"Bulstrode\" [JL].",
              "citations": [
                "G",
                "JL"
              ]
            },
            {
              "description": "Folder 3. Cucubalus baccifer [G]. Sheet 1. Cucubalus baccifer [JL]. i. \"Cucubalus bacciferus calyicibus campanulatis, petalis distantibus, per